http://mapfrappe.com/?show=51352

In [1]:
import time
import planarity
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import osmnx as ox
from shapely.geometry import Point, Polygon, LineString, MultiLineString

from planar_analysis import *

ox.config(use_cache=True, log_console=True)
%matplotlib inline

## Configure the script

In [2]:
df_places = pd.read_csv('data/places.csv')

distances = [805] #804.67 meters in each direction makes for 1 square mile, 500 meters is 1 sq km

network_types = ['drive', 'walk']

In [3]:
#df_places = df_places.loc[0:6]
df_places

,country,city,lat,lng
0,Argentina,Buenos Aires,-34.608657,-58.375547
1,Australia,Sydney,-33.863616,151.208977
2,Brazil,Sao Paulo,-23.549155,-46.633600
3,Canada,Toronto,43.649903,-79.380838
4,Canada,Vancouver,49.281651,-123.121363
5,Chile,Santiago,-33.439754,-70.654296
6,China,Beijing,39.908427,116.465505
7,China,Shanghai,31.236286,121.503787
8,China,Hong Kong,22.282321,114.156843
9,Denmark,Copenhagen,55.679443,12.578284


## Execute the script

In [4]:
# convert df to dict for execution
places = {}
for label, row in df_places.iterrows():
    key = '{}_{}'.format(row['country'], row['city'])
    value = (row['lat'], row['lng'])
    places[key] = value
    
places = df_places

In [5]:
results = []
for label, row in df_places.iterrows():
    
    coords = (row['lat'], row['lng'])
    point = Point(row['lng'], row['lat'])
    
    for network_type in network_types:
        
        for distance in distances:

            print(row['country'], row['city'], network_type, distance, end=' ')
            try:

                start_time = time.time()
                G = get_graph(coords, distance, network_type)

                # get a special buffered graph for planar analysis
                buffer = 500
                G_buff = get_graph(coords, distance + buffer, network_type)

                # get a bounding box to trim things square
                north, south, east, west = ox.bbox_from_point(coords, distance)
                bbox = Polygon([(west, north), (west, south), (east, south), (east, north)])

                # how many planar line intersections are there?
                planar_intersections = calculate_planar_intersections(G_buff, bbox)
                count_planar_intersections = len(planar_intersections)

                # how many nonplanar graph edge intersections are there?
                nonplanar_intersections = calculate_nonplanar_intersections(G, bbox)
                count_nonplanar_intersections = len(nonplanar_intersections)

                # how many cleaned, clustered intersections are there?
                cleaned_intersections = calculate_cleaned_intersections(nonplanar_intersections, G.graph['crs'])
                count_cleaned_intersections = len(cleaned_intersections)
                
                # how does planarity affect average edge length?
                mean_edge_length, mean_planar_segment_length, edge_length_ratio = calculate_edge_length_ratios(G, planar_intersections)

                if count_nonplanar_intersections > 0:
                    # planar line intersections overcounts nonplanar graph edge intersections by xx%
                    # ie, planar graph shows xx% more intersections than nonplanar graph with bridges/tunnels
                    planar_nonplanar_overcount = count_planar_intersections / count_nonplanar_intersections
                else:
                    planar_nonplanar_overcount = None #avoid divide by zero errors

                if count_cleaned_intersections > 0:
                    # edge intersections overcounts street intersections by xx%
                    nonplanar_cleaned_overcount = count_nonplanar_intersections / count_cleaned_intersections

                    # line intersections overcounts street intersections by xx%
                    planar_cleaned_overcount = count_planar_intersections / count_cleaned_intersections
                else:
                    planar_cleaned_overcount = None #avoid divide by zero errors
                    nonplanar_cleaned_overcount = None

                # is it a formally planar graph (ignoring spatial embedding)?
                warnings.filterwarnings('ignore')
                is_planar = planarity.is_planar(G)
                warnings.resetwarnings()
                
                runtime = round(time.time() - start_time, 2)
                print(runtime)
                
                # assemble the results
                result = {'country' : row['country'], 
                          'city' : row['city'],
                          'geometry' : point,
                          'distance' : distance,
                          'network_type' : network_type,
                          'nodes' : len(G.nodes()),
                          'count_planar_intersections' : count_planar_intersections,
                          'count_nonplanar_intersections' : count_nonplanar_intersections,
                          'count_cleaned_intersections' : count_cleaned_intersections,
                          'overcount_planar_nonplanar' : planar_nonplanar_overcount,
                          'overcount_nonplanar_cleaned' : nonplanar_cleaned_overcount,
                          'overcount_planar_cleaned' : planar_cleaned_overcount,
                          'mean_edge_length' : mean_edge_length,
                          'mean_planar_segment_length' : mean_planar_segment_length,
                          'edge_length_ratio' : edge_length_ratio,
                          'phi' : 1 / planar_nonplanar_overcount,
                          'is_planar' : is_planar,
                          'runtime' : runtime}
                results.append(result)

            except Exception as e:
                print(e)

Argentina Buenos Aires drive 805 9.08
Argentina Buenos Aires walk 805 67.44
Australia Sydney drive 805 5.79
Australia Sydney walk 805 81.5
Brazil Sao Paulo drive 805 15.17
Brazil Sao Paulo walk 805 46.64
Canada Toronto drive 805 8.4
Canada Toronto walk 805 1058.58
Canada Vancouver drive 805 6.55
Canada Vancouver walk 805 64.75
Chile Santiago drive 805 8.64
Chile Santiago walk 805 29.57
China Beijing drive 805 2.77
China Beijing walk 805 7.01
China Shanghai drive 805 5.29
China Shanghai walk 805 16.77
China Hong Kong drive 805 14.13
China Hong Kong walk 805 143.48
Denmark Copenhagen drive 805 9.82
Denmark Copenhagen walk 805 93.02
Egypt Cairo drive 805 12.59
Egypt Cairo walk 805 23.24
France Lyon drive 805 22.45
France Lyon walk 805 111.28
France Paris drive 805 14.24
France Paris walk 805 102.02
Germany Berlin drive 805 6.23
Germany Berlin walk 805 199.02
India Delhi drive 805 2.2
India Delhi walk 805 12.34
Indonesia Jakarta drive 805 12.14
Indonesia Jakarta walk 805 26.21
Iran Tehran 

In [6]:
gdf = gpd.GeoDataFrame(results, geometry='geometry').round(3)
gdf.crs = {'init': 'epsg:4326'}
print('finished in {:.2f} minutes'.format(gdf['runtime'].sum() / 60))

finished in 119.43 minutes


## Save results to disk

In [7]:
ox.save_graph_shapefile(G, filename='graph', folder='data')

In [8]:
planar_intersections.to_file('data/graph/planar')
nonplanar_intersections.to_file('data/graph/nonplanar')
cleaned_intersections.to_file('data/graph/cleaned')

In [9]:
north, south, east, west = ox.bbox_from_point(coords, distance)
polygon = Polygon([(west, north), (west, south), (east, south), (east, north)])
gpd.GeoSeries([polygon]).to_file('data/graph/bbox')

In [10]:
gdf.to_csv('data/results.csv', index=True, encoding='utf-8')

In [11]:
gdf2 = gdf.set_index(['country', 'city', 'distance', 'network_type'])
gdf2.iloc[gdf2.index.get_level_values('network_type') == 'drive'].to_csv('data/results_drive.csv')
gdf2.iloc[gdf2.index.get_level_values('network_type') == 'walk'].to_csv('data/results_walk.csv')